In [ ]:
!git clone https://github.com/leimao/Two_Layer_Hierarchical_Softmax_PyTorch

In [60]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data

import data
path = './data/ptb'
corpus = data.Corpus(path)

eval_batch_size = 10
train_data = batchify(corpus.train, 128)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

def get_batch(source, i, evaluation=False):
    seq_len = min(10, len(source) - 1 - i)
    data = Variable(source[i:i+seq_len], volatile=evaluation)
    target = Variable(source[i+1:i+1+seq_len].view(-1))
    return data, target

def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

def evaluate(model, data_source):
    # Turn on evaluation mode which disables dropout.
    #encoder.eval()
    model.eval()
    #decoder.eval()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    for i in range(0, data_source.size(0) - 1, 10):
        data, targets = get_batch(data_source, i, evaluation=True)
        emb = encoder(data)
        output, hidden = model(emb, hidden)

        probs = hierarchical_softmax(output.view(-1, output.size(2)), targets)

        loss = -torch.mean(torch.log(probs))

        total_loss += len(data) * loss.data

        hidden = repackage_hidden(hidden)
    return total_loss[0] / len(data_source)

In [10]:
evaluate(val_data)

4.832055081514371

In [2]:
import argparse
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

# import data
# import model
# import utils

In [3]:
model_saved = torch.load('model.pt')

/home/annakuzmina/anaconda3/lib/python3.5/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.GRU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/annakuzmina/anaconda3/lib/python3.5/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [63]:
model = model_saved['model']
encoder = model_saved['encoder']
hierarchical_softmax = model_saved['decoder']

In [64]:
model.add_module("encoder", encoder)
model.add_module("decoder", hierarchical_softmax)

In [ ]:
def SVDSoftmax(A, h, b, W_size, N_size, B, V_t):
    h = torch.mm(V_t,h)
    z = torch.zeros_like(b)
    torch.add(torch.mm(B[:, :W_size], h[:W_size]), b, out=z)

    top_k_ind = torch.topk(z, k=N_size, dim=0, sorted=False)[1][:,0]
    torch.add(torch.mm(B[top_k_ind], h), b[top_k_ind], out=z[top_k_ind])
    z_exp = torch.exp(z - torch.max(z))
    return z_exp / z_exp.sum()

In [9]:
import utils

In [10]:
utils.HierarchicalSoftmax

utils.HierarchicalSoftmax

In [ ]:
torch.bmm()

In [ ]:
def SVDSoftmax(A, h, b, W_size, N_size, B, V_t):
    h = torch.matmul(V_t, h)
    z = torch.zeros_like(b)
    torch.add(torch.matmul(B[:, :W_size], h[:W_size]), b, out=z)

    top_k_ind = torch.topk(z, k=N_size, dim=0, sorted=False)[1][:,0]
    torch.add(torch.matmul(B[top_k_ind], h), b[top_k_ind], out=z[top_k_ind])
    z_exp = torch.exp(z - torch.max(z))
    return z_exp / z_exp.sum()

In [ ]:
batch_size, d = inputs.size()
label_position_top = labels / self.ntokens_per_class
label_position_bottom = labels % self.ntokens_per_class

layer_top_logits = torch.matmul(inputs, self.layer_top_W) + self.layer_top_b
layer_top_probs = self.softmax(layer_top_logits)

In [157]:
class SVDSoftmax(nn.Module):
    def __init__(self, weights, bias):
        super(SVDSoftmax, self).__init__()
        self.A = weights
        self.W_size, self.N_size = int(self.A.shape[1] / 8), int(self.A.shape[0] / 10)
        self.b = bias
        
        U, S, V = torch.svd(weights)
        self.V_t = V.transpose(0, 1)
        self.B = U * S
    
    def forward(self, inputs):
        h = torch.matmul(self.V_t, inputs)
        z = torch.zeros_like(self.b)
        print(h.shape, z.shape, self.B.shape)
        z = torch.matmul(self.B[:, :self.W_size], h[:self.W_size])
        z = z + self.b
#         torch.add(torch.mm(self.B.data[:, :self.W_size], h[:self.W_size]), self.b.data, out=z.data)

        top_k_ind = torch.topk(z, k=self.N_size, dim=0, sorted=False)[1][:,0]
        torch.add(torch.mm(self.B[top_k_ind], h), self.b[top_k_ind], out=z[top_k_ind])
        z_exp = torch.exp(z - torch.max(z, dim=-1))
        return z_exp / z_exp.sum(dim=-1)
        

In [148]:
class HierarchicalSoftmaxWithSVD(utils.HierarchicalSoftmax):
    def __init__(self, pretrained_hierarchical_softmax):
        super(HierarchicalSoftmaxWithSVD, self).__init__(pretrained_hierarchical_softmax.ntokens,
                                                         pretrained_hierarchical_softmax.nhid,
                                                         pretrained_hierarchical_softmax.ntokens_per_class)
    
        
        self.load_state_dict(pretrained_hierarchical_softmax.state_dict())        
        self.svd_softmax_top_W = SVDSoftmax(self.layer_top_W, self.layer_top_b)
        
    def forward(self, inputs, labels = None):

        batch_size, d = inputs.size()

        if labels is not None:
            

            label_position_top = labels / self.ntokens_per_class
            label_position_bottom = labels % self.ntokens_per_class
            
#             layer_top_logits = torch.matmul(inputs, self.layer_top_W) + self.layer_top_b
#             layer_top_probs = self.softmax(layer_top_logits)
            print("SVD")
            layer_top_probs = self.svd_softmax_top_W(inputs)

            layer_bottom_logits = torch.squeeze(torch.bmm(torch.unsqueeze(inputs, dim=1), self.layer_bottom_W[label_position_top]), dim=1) + self.layer_bottom_b[label_position_top]
            layer_bottom_probs = self.softmax(layer_bottom_logits)

            target_probs = layer_top_probs[torch.arange(batch_size).long(), label_position_top] * layer_bottom_probs[torch.arange(batch_size).long(), label_position_bottom]

            return target_probs

        else:
            print("NO SVD")
            # Remain to be implemented
            layer_top_logits = torch.matmul(inputs, self.layer_top_W) + self.layer_top_b
            layer_top_probs = self.softmax(layer_top_logits)

            word_probs = layer_top_probs[:,0] * self.softmax(torch.matmul(inputs, self.layer_bottom_W[0]) + self.layer_bottom_b[0])

            for i in range(1, self.nclasses):
                word_probs = torch.cat((word_probs, layer_top_probs[:,i] * self.softmax(torch.matmul(inputs, self.layer_bottom_W[i]) + self.layer_bottom_b[i])), dim=1)

            return word_probs
        
        

#         self.init_weights(hierarchical_softmax.parameters())

In [163]:
hierarchical_softmax.l

torch.Size([150, 100])

In [158]:
svd_decoder = HierarchicalSoftmaxWithSVD(hierarchical_softmax)

In [133]:
hidden = svd_model.init_hidden(eval_batch_size)
data, targets = get_batch(val_data, 0, evaluation=True)
emb = encoder(data)
output, hidden = svd_model(emb, hidden)

In [129]:
svd_model = model_saved['model']

In [130]:
svd_model.add_module("encoder", encoder)
svd_model.add_module("decoder", svd_decoder)

In [131]:
svd_model

RNNModel(
  (drop): Dropout(p=0.0)
  (rnn): GRU(100, 150)
  (encoder): Word2VecEncoder(
    (drop): Dropout(p=0.0)
    (encoder): Embedding(10000, 100)
  )
  (decoder): HierarchicalSoftmaxWithSVD(
    (softmax): Softmax()
    (svd_softmax_top_W): SVDSoftmax(
    )
  )
)

In [76]:
def evaluate_svd(data_source):
    # Turn on evaluation mode which disables dropout.
    #encoder.eval()
    svd_model.eval()
    #decoder.eval()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    hidden = svd_model.init_hidden(eval_batch_size)
    for i in range(0, data_source.size(0) - 1, 10):
        data, targets = get_batch(data_source, i, evaluation=True)
        emb = encoder(data)
        output, hidden = svd_model(emb, hidden)

        probs = svd_decoder(output.view(-1, output.size(2)), targets)

        loss = -torch.mean(torch.log(probs))

        total_loss += len(data) * loss.data

        hidden = repackage_hidden(hidden)
    return total_loss[0] / len(data_source)